TODO
1. 시장별 시가 확인
2. 시장별 메뉴 가격 확인
3. 상관관계 분석

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib
from datetime import datetime, timedelta

In [100]:
file_path = "../data/raw/ikh_item_price_2025-01-14.csv"
df = pd.read_csv(file_path)
df["priceDate"] = pd.to_datetime(df["priceDate"])
df_market_item_price = df[df["priceDate"] == "2025-01-14"][['market', 'item', 'avgPrice']]
df_market_item_price

,market,item,avgPrice
3188,노량진 1층,농어,37250
6395,가락시장,농어,43000
8559,노량진 2층,농어,36430
11432,구리농수산물시장,농어,42500
14384,강서농수산물시장,농어,45000
...,...,...,...
131036,노량진 2층,방어,54000
131189,구리농수산물시장,방어,48750
131460,수원농수산물시장,방어,50000
131524,인천종합연안부두어시장,방어,35000


In [92]:
df_market_item_price[df_market_item_price['item'] == '광어']
df_market_item_price['item'].unique()

array(['농어', '광어', '대게', '연어', '우럭', '참돔', '방어'], dtype=object)

In [23]:
file_path = "../data/processed/ikh_store_meta_2025-01-14.csv"
df = pd.read_csv(file_path)
df_market_store = df[['market', 'store_id']]
df_market_store

,market,store_id
0,노량진 1층,833
1,노량진 1층,5
2,노량진 2층,822
3,노량진 1층,13
4,노량진 1층,6
...,...,...
148,부산민락어민활어직판장,123
149,부산민락어민활어직판장,338
150,부산민락어민활어직판장,433
151,부산민락어민활어직판장,388


In [77]:
file_path = "../data/raw/ikh_store_menu_2025-01-14.csv"
df = pd.read_csv(file_path)
df_market_menu_price = df[['store_id', 'name', 'menu_id', 'menu']].copy()
df_market_menu_price['price'] = df['price'] - df['discount']
df_market_menu_price = pd.merge(df_market_store, df_market_menu_price, on="store_id", how="inner")
print(df_market_menu_price.shape)
df_market_menu_price.groupby(['name']).count().sort_values("store_id", ascending=False)['menu'].head(20)


(7668, 6)


name
대방어회         135
광우회          110
대광어회          89
도다리모둠회        86
광어연어회         79
광돔회           77
광연회           71
활모둠회          70
대방어모둠회        66
방어회           65
방어모둠회         62
방광회           60
연어회           58
특대방어회         56
도다리세꼬시모둠회     54
대방어광어회        53
광우연회          48
광참회           47
참광우회          46
참치모둠회         44
Name: menu, dtype: int64

In [71]:
df_market_menu_price = df_market_menu_price[
    df_market_menu_price['name'].str.contains('^농어|^광어|^대게|^연어|^우럭|^참돔|^방어', na=False, regex=True)
]

df_tmp = df_market_menu_price.groupby(['name', 'menu']).count().sort_values("store_id", ascending=False)
print(df_tmp)
set([i[0] for i in df_tmp[df_tmp["store_id"] > 3].index])

                 market  store_id  menu_id  price
name  menu                                       
광어연어회 중(2~3인)        21        21       21     21
      대(3~4인)        19        19       19     19
      소(2인)          19        19       19     19
연어회   대(3~4인)        16        16       16     16
      중(2~3인)        16        16       16     16
...                 ...       ...      ...    ...
연어꽃회  대(4인)           1         1        1      1
      대(5~6인)         1         1        1      1
      소(2~3인)         1         1        1      1
      중(2~3인)         1         1        1      1
참돔회   특특대(5-6인)       1         1        1      1

[314 rows x 4 columns]


{'광어연어회',
 '광어우럭회',
 '광어참돔회',
 '광어회',
 '농어모둠회',
 '농어회',
 '방어광어회',
 '방어연어회',
 '방어참돔회',
 '연어회',
 '우럭모둠회',
 '참돔모둠회',
 '참돔회'}

In [114]:
filtered_df = df_market_menu_price[
    df_market_menu_price['name'].isin(["광어회", "농어회", "연어회", "참돔회", "방어회"]) &
    df_market_menu_price['menu'].str.contains("^대", na=False)
]  # 대(3~4인) 기준
# filtered_df[['market', 'name', 'price']]
filtered_df[['market', 'name', 'price']].groupby(['name', 'market']).mean()['price']

name  market     
광어회   가락시장            85000.000000
      구리농수산물시장        80000.000000
      노량진 1층         102500.000000
      노량진 2층         100000.000000
농어회   구리농수산물시장       130000.000000
      노량진 1층         120000.000000
      노량진 2층          90000.000000
      부산민락어민활어직판장     70000.000000
방어회   가락시장           111000.000000
      강서농수산물시장       130000.000000
      구리농수산물시장        88333.333333
      노량진 1층         129000.000000
      노량진 2층         108000.000000
      부산민락어민활어직판장     54000.000000
연어회   가락시장            81666.666667
      강서농수산물시장        70000.000000
      구리농수산물시장        56666.666667
      노량진 1층          95000.000000
      노량진 2층          80000.000000
      수원농수산물시장        60000.000000
      안양평촌농수산물시장      70000.000000
      인천종합연안부두어시장     80000.000000
참돔회   구리농수산물시장        88333.333333
      노량진 1층          98750.000000
      노량진 2층         100000.000000
      수원농수산물시장        80000.000000
Name: price, dtype: float64

In [115]:
df_tmp = df_market_item_price.copy()
for item in df_tmp["item"].unique():
    df_tmp.loc[df_tmp["item"] == item, "name"] = item + "회"

In [116]:
# 두 데이터프레임 병합
merged_df = pd.merge(
    filtered_df[['market', 'name', 'price']].groupby(['name', 'market']).mean().reset_index(),
    df_tmp,
    on=['market', 'name'],
    how='inner'  # 내부 조인 (공통 데이터만 포함)
)[["name", "market", "price", "avgPrice"]]

# 결과 확인
print(merged_df)

   name       market          price  avgPrice
0   광어회         가락시장   85000.000000     45000
1   광어회     구리농수산물시장   80000.000000     40000
2   광어회       노량진 1층  102500.000000     39770
3   광어회       노량진 2층  100000.000000     40000
4   농어회     구리농수산물시장  130000.000000     42500
5   농어회       노량진 1층  120000.000000     37250
6   농어회       노량진 2층   90000.000000     36430
7   방어회         가락시장  111000.000000     47780
8   방어회     구리농수산물시장   88333.333333     48750
9   방어회       노량진 1층  129000.000000     47650
10  방어회       노량진 2층  108000.000000     54000
11  연어회         가락시장   81666.666667      6450
12  연어회     강서농수산물시장   70000.000000      6780
13  연어회     구리농수산물시장   56666.666667      5880
14  연어회       노량진 1층   95000.000000      6840
15  연어회       노량진 2층   80000.000000      6510
16  연어회     수원농수산물시장   60000.000000      6000
17  연어회  인천종합연안부두어시장   80000.000000      9000
18  참돔회     구리농수산물시장   88333.333333     43340
19  참돔회       노량진 1층   98750.000000     39400
20  참돔회       노량진 2층  100000.00000

In [117]:
from scipy.stats import pearsonr

# 상관계수를 저장할 결과 리스트
correlation_results = []

# 품목별로 그룹화하여 상관관계 계산
for name, group in merged_df.groupby('name'):
    # Pearson 상관계수 계산
    correlation, p_value = pearsonr(group['price'], group['avgPrice'])
    
    # 결과 저장
    correlation_results.append({'name': name, 'correlation': correlation, 'p_value': p_value})

# 결과를 데이터프레임으로 변환
correlation_df = pd.DataFrame(correlation_results)

# 결과 출력
print(correlation_df)

  name  correlation   p_value
0  광어회    -0.436636  0.563364
1  농어회     0.777686  0.432786
2  방어회    -0.195843  0.804157
3  연어회     0.439887  0.323327
4  참돔회    -0.561916  0.438084
